## Project Birds of Ireland

# Project Objectives:

To visualize the geographic distribution and concentration of bird populations across Ireland.

To develop an interactive map that enables users to explore bird concentration patterns spatially.

To implement filtering capabilities based on year, month, and bird species, allowing for flexible temporal and taxonomic analysis.

This analysis is based on bird observation data downloaded from the **Irish Spatial Data Exchange (ISDE)** on August 6, 2025.
https://www.isde.ie/geonetwork/srv/eng/catalog.search#/metadata/ie.nbdc.dataset.BirdsOfIreland

The dataset covers the period from January 1, 1970, with the majority of records collected from 2011 onwards.

As the dataset is ongoing, this project is designed to be extensible and can be updated with new data in future releases.

In [1]:
# Libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [4]:
birds = pd.read_csv("BirdsOfIreland.csv", delimiter='\t')
birds.head()

<ipython-input-4-969c5018fb82>:1: DtypeWarning: Columns (0,1,12,16,17,18,20,21,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  birds = pd.read_csv("BirdsOfIreland.csv", delimiter='\t')


,SurveyKey,SampleKey,StartDate,EndDate,DateType,Date,TaxonVersionKey,TaxonName,GridReference,East,...,"Habitat code (Fossitt, 2000)",Type of sighting,Activity,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,NaN,NaN,13/07/2019,13/07/2019,D,13/07/2019,NBNSYS0000001346,Motacilla alba yarrellii,S8241,282500.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,23/05/2020,23/05/2020,D,23/05/2020,NBNSYS0000001346,Motacilla alba yarrellii,O311082,331150.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,11/05/2021,11/05/2021,D,11/05/2021,NBNSYS0000001346,Motacilla alba yarrellii,NaN,-6.185258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,26/05/2021,26/05/2021,D,26/05/2021,NBNSYS0000001346,Motacilla alba yarrellii,NaN,-6.402222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,01/01/2021,01/01/2021,D,01/01/2021,NBNSYS0000001346,Motacilla alba yarrellii,N406445,240650.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
birds.shape

(278634, 40)

In [6]:
birds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278634 entries, 0 to 278633
Data columns (total 40 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   SurveyKey                     22771 non-null   object 
 1   SampleKey                     12 non-null      object 
 2   StartDate                     278023 non-null  object 
 3   EndDate                       278011 non-null  object 
 4   DateType                      278003 non-null  object 
 5   Date                          278003 non-null  object 
 6   TaxonVersionKey               278015 non-null  object 
 7   TaxonName                     278011 non-null  object 
 8   GridReference                 172742 non-null  object 
 9   East                          278005 non-null  float64
 10  North                         278003 non-null  float64
 11  Projection                    278004 non-null  object 
 12  Precision                     278010 non-nul

### Data Preprocessing Notes:

During initial inspection of the dataset using LibreOffice Calc, several unnamed columns were identified. Upon further examination, it became evident that these columns likely resulted from text fields containing the CSV delimiter (commas) that were not properly enclosed in quotes during export.

These fragmented columns appeared to contain parts of the same textual field, specifically related to the "Activity" column. To address this, all unnamed columns were merged with the "Activity" column to reconstruct the complete text field and preserve the integrity of the original data.

This step ensures accurate parsing of the dataset and prevents loss of contextual information related to bird observation activities.

In [7]:
cols_to_merge = ['Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39']
for col in cols_to_merge:
    birds['Activity'] = birds['Activity'].fillna(birds[col])

birds = birds.drop(columns=cols_to_merge)

display(birds[['Activity']].head())

,Activity
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [8]:
birds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278634 entries, 0 to 278633
Data columns (total 33 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   SurveyKey                     22771 non-null   object 
 1   SampleKey                     12 non-null      object 
 2   StartDate                     278023 non-null  object 
 3   EndDate                       278011 non-null  object 
 4   DateType                      278003 non-null  object 
 5   Date                          278003 non-null  object 
 6   TaxonVersionKey               278015 non-null  object 
 7   TaxonName                     278011 non-null  object 
 8   GridReference                 172742 non-null  object 
 9   East                          278005 non-null  float64
 10  North                         278003 non-null  float64
 11  Projection                    278004 non-null  object 
 12  Precision                     278010 non-nul

**Date Field Validation and Imputation Strategy:**

The dataset contains two relevant temporal fields: date and startDate.

date has 278,003 non-null entries.

startDate has 278,023 non-null entries — 20 more than date.

To maximize the temporal coverage of the dataset, the 20 entries where startDate is available but date is null will be reviewed to determine if startDate can be reliably used as a proxy for date.

If confirmed consistent, these values will be imputed into the date field to enhance completeness and ensure uniformity for time-based filtering and analysis.

In [9]:
display(birds[birds['Date'].isnull() & birds['StartDate'].notnull()])

,SurveyKey,SampleKey,StartDate,EndDate,DateType,Date,TaxonVersionKey,TaxonName,GridReference,East,...,Habitat description,Abundance,Determiner name,Survey name,Common name,Site description,Source,"Habitat code (Fossitt, 2000)",Type of sighting,Activity
55754,NaN,False,False,NaN,NaN,NaN,Mayo,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61441,NaN,False,False,NaN,NaN,NaN,Mayo,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66834,Ben Malone,NaN,False,False,NaN,NaN,NaN,Galway,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81735,Brendan McSherry,NaN,False,False,NaN,NaN,NaN,Donegal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81861,Brendan McSherry,NaN,False,False,NaN,NaN,NaN,Donegal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83148,Brendan McSherry,NaN,False,False,NaN,NaN,NaN,Donegal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86397,Brendan McSherry,NaN,False,False,NaN,NaN,NaN,Donegal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87502,Anthony Dawson,NaN,False,False,NaN,NaN,NaN,Galway,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87653,Brendan McSherry,NaN,False,False,NaN,NaN,NaN,Donegal,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138625,NaN,False,False,NaN,NaN,NaN,Laois,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The startDate field cannot be used to supplement missing values in the date field due to invalid content. 

**Initial Feature Selection and Field Removal**

As part of the data preprocessing phase, an initial review of all fields was conducted to identify columns that can be removed due to irrelevance, redundancy, or poor data quality. The goal is to streamline the dataset for analysis and visualization by retaining only meaningful and reliable features.

**Criteria for Removal:**

Empty or Nearly Empty Fields: Columns with high proportions of null or placeholder values.

Duplicate or Redundant Fields: Columns that replicate information available elsewhere.

Corrupted or Invalid Data: Fields (e.g., startDate) containing non-informative or clearly erroneous values (e.g., "false").

Administrative or Irrelevant Metadata: Internal processing fields or identifiers not required for analysis (e.g., system-generated IDs, timestamps unrelated to observations).

Non-standardized Text Fragments: Unnamed or fragmented columns caused by CSV parsing issues (e.g., misplaced delimiters), which were addressed separately (e.g., merging into the Activity field).


In [11]:
fieldsAnalysis = pd.read_csv("fieldsAnalysis.csv", delimiter=';')
fieldsAnalysis

,#,Column,non-Null,CountDtype,to_be_removed,analysis
0,0,SurveyKey,22771 non-Null,object,*,identifiers not required for analysis
1,1,SampleKey,12 non-Null,object,*,Empty or Nearly Empty Field
2,2,StartDate,278023 non-Null,object,*,"identifiers not required for analysis, using D..."
3,3,EndDate,278011 non-Null,object,*,"identifiers not required for analysis, using D..."
4,4,DateType,278003 non-Null,object,,This field will be used to determine the level...
5,5,Date,278003 non-Null,object,,Effective for data filtering purposes.
6,6,TaxonVersionKey,278015 non-Null,object,,Effective for bird identification.
7,7,TaxonName,278011 non-Null,object,,Effective for bird identification.
8,8,GridReference,172742 non-Null,object,,Offers a detailed explanation for reference.
9,9,East,278005 non-Null,float64,,This field will be used to determine the bird'...


In [12]:
# removing fields: 
cols_to_drop = ['SurveyKey', 'SampleKey', 'StartDate', 'EndDate', 'Recorder', 'Determiner', 'ZeroAbundance', 'UnderValidation', 'Determiner name', 'Survey name', 'Site description', 'Source', 'Habitat code (Fossitt, 2000)', 'SiteKey', 'ImagePath']
birds = birds.drop(columns=cols_to_drop)
display(birds.head())

,DateType,Date,TaxonVersionKey,TaxonName,GridReference,East,North,Projection,Precision,SiteName,Record comment,Vice-county,County,Habitat description,Abundance,Common name,Type of sighting,Activity
0,D,13/07/2019,NBNSYS0000001346,Motacilla alba yarrellii,S8241,282500.000000,141500.000000,OSI,1000,Monksgrange,NaN,NaN,Wexford,NaN,NaN,Pied Wagtail,NaN,NaN
1,D,23/05/2020,NBNSYS0000001346,Motacilla alba yarrellii,O311082,331150.000000,208250.000000,OSI,100,Kilcoole,NaN,NaN,Wicklow,NaN,NaN,Pied Wagtail,NaN,NaN
2,D,11/05/2021,NBNSYS0000001346,Motacilla alba yarrellii,NaN,-6.185258,54.036502,WGS84,100,Carlingford Carlingford,NaN,NaN,Louth,Built land,NaN,Pied Wagtail,NaN,NaN
3,D,26/05/2021,NBNSYS0000001346,Motacilla alba yarrellii,NaN,-6.402222,54.008375,WGS84,100,"St Nicholas Quarter, Dundalk",NaN,NaN,Louth,Built land,2,Pied Wagtail,NaN,NaN
4,D,01/01/2021,NBNSYS0000001346,Motacilla alba yarrellii,N406445,240650.000000,244550.000000,OSI,100,Robinstown,NaN,NaN,Westmeath,NaN,2,Pied Wagtail,NaN,NaN


In [13]:
birds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278634 entries, 0 to 278633
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   DateType             278003 non-null  object 
 1   Date                 278003 non-null  object 
 2   TaxonVersionKey      278015 non-null  object 
 3   TaxonName            278011 non-null  object 
 4   GridReference        172742 non-null  object 
 5   East                 278005 non-null  float64
 6   North                278003 non-null  float64
 7   Projection           278004 non-null  object 
 8   Precision            278010 non-null  object 
 9   SiteName             278002 non-null  object 
 10  Record comment       10910 non-null   object 
 11  Vice-county          21776 non-null   object 
 12  County               255547 non-null  object 
 13  Habitat description  5772 non-null    object 
 14  Abundance            183666 non-null  object 
 15  Common name      

To ensure consistency in spatial referencing, all records must use a unified coordinate system. We will create latitude and longitude fields. For records where the projection is 'WGS84', the North and East values will be directly assigned to latitude and longitude, respectively. For records using the 'OSI' projection, the North and East coordinates will be transformed from the Irish Grid (OSI) to the WGS84 coordinate system before populating the latitude and longitude fields.

In [18]:
# install the library
%pip install pyproj

In [20]:
from pyproj import Transformer

def osi_to_wgs84(east, north):
    """Converts Irish Grid (OSI) coordinates to WGS84 latitude and longitude."""
    # Define the transformers
    osi = 'epsg:29902'  # Irish Grid (OSI)
    wgs84 = 'epsg:4326' # WGS84

    transformer = Transformer.from_crs(osi, wgs84)

    # Perform the transformation
    lon, lat = transformer.transform(east, north)
    return lat, lon

In [22]:
# Create latitude and longitude fields
birds['latitude'] = None
birds['longitude'] = None

# Copy values for WGS84
wgs_mask = birds['Projection'] == 'WGS84'
birds.loc[wgs_mask, 'latitude'] = birds.loc[wgs_mask, 'North']
birds.loc[wgs_mask, 'longitude'] = birds.loc[wgs_mask, 'East']

# Convert and populate for OSI
osi_mask = birds['Projection'] == 'OSI'
converted_coords = birds.loc[osi_mask, ['East', 'North']].apply(
    lambda row: osi_to_wgs84(row['East'], row['North']), axis=1, result_type='expand'
)

# Assign the converted latitude and longitude to the correct columns
birds.loc[osi_mask, 'longitude'] = converted_coords[0]
birds.loc[osi_mask, 'latitude']  = converted_coords[1]

# Display the updated DataFrame with the new columns
display(birds[['East', 'North', 'Projection', 'latitude', 'longitude']].head())

,East,North,Projection,latitude,longitude
0,282500.000000,141500.000000,OSI,52.519049,-6.785386
1,331150.000000,208250.000000,OSI,53.108982,-6.042302
2,-6.185258,54.036502,WGS84,54.036502,-6.185258
3,-6.402222,54.008375,WGS84,54.008375,-6.402222
4,240650.000000,244550.000000,OSI,53.44968,-7.388904


**Atention:** For larger datasets, this code may require optimization, as its current implementation results in extended execution time.

In [23]:
# Proceeding to inspect and count the unique values present in the dataTypes column.
birds['DateType'].value_counts()

DateType
D    277945
Y        33
O        25
Name: count, dtype: int64

In [24]:
# Identified 25 records with dataTypes set to 'O'; proceeding to display them for analysis.
display(birds[birds['DateType'] == 'O'])

,DateType,Date,TaxonVersionKey,TaxonName,GridReference,East,North,Projection,Precision,SiteName,Record comment,Vice-county,County,Habitat description,Abundance,Common name,Type of sighting,Activity,latitude,longitude
2882,O,Jul 2007,NHMSYS0000530133,Aegithalos caudatus,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
2911,O,Jul 2007,NHMSYS0000530314,Corvus cornix,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,Kildare,Kildare,NaN,Cutaway - pioneer vegetation,Cutaway - pioneer vegetation,53.316211,-6.875101
3643,O,Jul 2007,NHMSYS0000530314,Corvus cornix,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,Kildare,Kildare,NaN,Cutaway - pioneer vegetation,Cutaway - pioneer vegetation,53.316211,-6.875101
3647,O,Jul 2007,NBNSYS0000000007,Anas platyrhynchos,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
6372,O,Jul 2007,NHMSYS0000530314,Corvus cornix,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,Kildare,Kildare,NaN,Cutaway - pioneer vegetation,Cutaway - pioneer vegetation,53.316211,-6.875101
9161,O,Jul 2007,NHMSYS0021004312,Acanthis cabaret,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
9162,O,Jul 2007,NHMSYS0000530739,Erithacus rubecula,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
11182,O,Jul 2007,NBNSYS0000000131,Anas crecca,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
11559,O,Jul 2007,NHMSYS0000530803,Rallus aquaticus,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
11960,O,Nov 2010,NHMSYS0000530685,Tyto alba,M92043551,192045.000000,235515.000000,OSI,10,Clonboley1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,53.370006,-8.120246


In [25]:
# There are 24 records with the value 'Jul 2007' and one record with 'Nov 2010'.
birds['Date'][birds['DateType'] == 'O'].value_counts()

Date
Jul 2007    24
Nov 2010     1
Name: count, dtype: int64

In [26]:
# These dates can be standardized by assigning the 15th day of the month, as this approximation does not impact the outcomes of this project.
birds.loc[(birds['Date'] == 'Jul 2007') & (birds['DateType'] == 'O'), 'Date'] = '15/07/2007'
birds.loc[(birds['Date'] == 'Nov 2010') & (birds['DateType'] == 'O'), 'Date'] = '15/11/2010'
display(birds[birds['DateType'] == 'O'])

,DateType,Date,TaxonVersionKey,TaxonName,GridReference,East,North,Projection,Precision,SiteName,Record comment,Vice-county,County,Habitat description,Abundance,Common name,Type of sighting,Activity,latitude,longitude
2882,O,15/07/2007,NHMSYS0000530133,Aegithalos caudatus,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
2911,O,15/07/2007,NHMSYS0000530314,Corvus cornix,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,Kildare,Kildare,NaN,Cutaway - pioneer vegetation,Cutaway - pioneer vegetation,53.316211,-6.875101
3643,O,15/07/2007,NHMSYS0000530314,Corvus cornix,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,Kildare,Kildare,NaN,Cutaway - pioneer vegetation,Cutaway - pioneer vegetation,53.316211,-6.875101
3647,O,15/07/2007,NBNSYS0000000007,Anas platyrhynchos,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
6372,O,15/07/2007,NHMSYS0000530314,Corvus cornix,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,Kildare,Kildare,NaN,Cutaway - pioneer vegetation,Cutaway - pioneer vegetation,53.316211,-6.875101
9161,O,15/07/2007,NHMSYS0021004312,Acanthis cabaret,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
9162,O,15/07/2007,NHMSYS0000530739,Erithacus rubecula,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
11182,O,15/07/2007,NBNSYS0000000131,Anas crecca,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
11559,O,15/07/2007,NHMSYS0000530803,Rallus aquaticus,NaN,-6.875101,53.316211,WGS84,100,Timahoe South,NaN,NaN,Kildare,NaN,NaN,NaN,NaN,NaN,53.316211,-6.875101
11960,O,15/11/2010,NHMSYS0000530685,Tyto alba,M92043551,192045.000000,235515.000000,OSI,10,Clonboley1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,53.370006,-8.120246


In [27]:
# inspect the dataset for null values
display(birds.isnull().sum())

DateType                  631
Date                      631
TaxonVersionKey           619
TaxonName                 623
GridReference          105892
East                      629
North                     631
Projection                630
Precision                 624
SiteName                  632
Record comment         267724
Vice-county            256858
County                  23087
Habitat description    272862
Abundance               94968
Common name            167075
Type of sighting       255881
Activity               227613
latitude                  631
longitude                 631
dtype: int64

In [28]:
# Removig rows from the dataset where the latitude field contains null values.
birds.dropna(subset=['latitude'], inplace=True)
display(birds.isnull().sum())

DateType                    0
Date                        0
TaxonVersionKey             0
TaxonName                   0
GridReference          105270
East                        0
North                       0
Projection                  0
Precision                   0
SiteName                    1
Record comment         267093
Vice-county            256227
County                  22456
Habitat description    272231
Abundance               94337
Common name            166444
Type of sighting       255250
Activity               226982
latitude                    0
longitude                   0
dtype: int64

In [29]:
# Replacing missing commonName entries by referencing non-null values from rows sharing the same taxonName.
# Create a mapping from TaxonName to the first non-null Common name
# Use .transform('first') to get the first non-null value in each group
taxon_to_common = birds.groupby('TaxonName')['Common name'].transform('first')

# Fill the null values in 'Common name' using this mapping
birds['Common name'] = birds['Common name'].fillna(taxon_to_common)

# Display the updated null counts for 'Common name' to verify
display(birds['Common name'].isnull().sum())

np.int64(32)

In [30]:
display(birds[birds['Common name'].isnull() & birds['TaxonName'].notnull()])

,DateType,Date,TaxonVersionKey,TaxonName,GridReference,East,North,Projection,Precision,SiteName,Record comment,Vice-county,County,Habitat description,Abundance,Common name,Type of sighting,Activity,latitude,longitude
36977,D,01/10/2015,NHMSYS0000530688,Vanellus gregarius,H896493,289650.000000,349350.000000,OSI,100,Ballybrannon,NaN,Armagh,Armagh,Cultivated land,2,None,NaN,Singing male,54.3849,-6.620871
37551,D,28/08/2014,NHMSYS0021537818,Hydrobates monorhis,Q658324,65850.000000,132450.000000,OSI,100,Kerry Head,NaN,North Kerry,Kerry,NaN,1,None,Flying over,NaN,52.427506,-9.972919
108563,D,27/06/2020,NHMSYS0020303142,Phasianus colchicus colchicus,F708089,70850.000000,308950.000000,OSI,100,Achill,NaN,NaN,Mayo,NaN,NaN,None,NaN,NaN,54.013689,-9.970917
179029,D,11/07/2020,NBNSYS0000188694,Pluvialis fulva,Q962664,96250.000000,166450.000000,OSI,100.0,Doonbeg,NaN,NaN,Clare,NaN,1.0,None,NaN,Summering non-breeder,52.739536,-9.536841
182314,D,20/05/2020,NHMSYS0000530145,Alopochen aegyptiaca,H712201,271250.000000,320150.000000,OSI,100,Ballybay,NaN,NaN,Monaghan,NaN,NaN,None,NaN,NaN,54.125529,-6.910951
184889,D,27/07/2006,NHMSYS0000534047,Threskiornis aethiopicus,W703721,170350.000000,72150.000000,OSI,100,"The Atlantic Pond, Cork",Probably escaped from Fota Wildlife Park.,NaN,Cork,NaN,1.0,None,NaN,NaN,51.901267,-8.431489
186935,D,02/07/2020,NBNSYS0000188694,Pluvialis fulva,Q990695,99050.000000,169550.000000,OSI,100,Doonbeg,NaN,NaN,Clare,NaN,NaN,None,NaN,NaN,52.767912,-9.496354
188566,D,17/04/2020,NBNSYS0000000276,Bucephala islandica,NaN,-6.265934,53.313160,WGS84,100,Dartry,NaN,NaN,Dublin,NaN,4.0,None,NaN,NaN,53.31316,-6.265934
196722,D,17/01/2014,NHMSYS0000533047,Larus glaucoides glaucoides,Q751202,75150.000000,120250.000000,OSI,100.0,Carrahane,NaN,NaN,Kerry,NaN,NaN,None,NaN,NaN,52.320129,-9.831736
196821,D,29/03/2019,NBNSYS0000000258,Anser erythropus,G670065,167050.000000,306550.000000,OSI,100.0,Greyfield,NaN,NaN,Sligo,NaN,15.0,None,NaN,NaN,54.007226,-8.503286
